In [230]:
from art.estimators.classification import BlackBoxClassifier
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.linear_model import LogisticRegression

X, y = load_breast_cancer(return_X_y=True)
X = X[:, 2:4]
clf = LogisticRegression(random_state = 0)

In [231]:
len(set(y))

2

In [232]:
from art.estimators.classification.scikitlearn import ScikitlearnClassifier

clf = ScikitlearnClassifier(model = clf)

In [233]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Create blackbox object
classifier = BlackBoxClassifier(clf.predict, X_train[0].shape, len(set(y)))

In [234]:
i = 1
target_instance = X[i]
target_instance, y[i]

(array([ 132.9, 1326. ]), 0)

## Generate black box HopSkipJump attack


In [235]:
import numpy as np
from art.attacks.evasion import HopSkipJump

# Generate HopSkipJump attack against black box classifier
attack = HopSkipJump(classifier=clf,
                     targeted=False,
                     max_iter=0,
                     max_eval=1000,
                     init_eval=10,
                     verbose = False
                    )
iter_step = 10
x_adv = None

print("Original instance: ", target_instance)

for i in range(10):
    
    x_adv = attack.generate(x=np.array([target_instance]), x_adv_init=x_adv)
    
    print(f"Adv. instance ", x_adv[0], ", step ", i * iter_step, " L2 error", 
          np.linalg.norm(np.reshape(x_adv[0] - target_instance, [-1])),
          "class: %d." % np.argmax(classifier.predict(x_adv)[0]))
    
    attack.max_iter = iter_step

TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [236]:
print(x_adv[0])
print(np.argmax(classifier.predict(x_adv)[0]))

[1.4 0.2]


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [237]:
print(target_instance)
print(np.argmax(classifier.predict(target_instance.reshape(1, -1))))

[ 132.9 1326. ]


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## Training scikit-learn RandomForestClassifier and attacking with ART Zeroth Order Optimization attack


### Random Forest

In [238]:
from sklearn.ensemble import RandomForestClassifier
from art.attacks.evasion import ZooAttack
from art.estimators.classification import SklearnClassifier

# Create and fit RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Create ART classifier for scikit-learn RandomForestClassifier
art_classifier = SklearnClassifier(model=model)

# Create ART Zeroth Order Optimization attack
zoo = ZooAttack(classifier=art_classifier, binary_search_steps=10, nb_parallel=1)

# Generate adversarial samples with ART Zeroth Order Optimization attack
x_train_adv = zoo.generate(X_test)

C:\Users\ivanr\anaconda3\envs\PhD\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


ZOO:   0%|          | 0/114 [00:00<?, ?it/s]

In [239]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
y_pred_adv = model.predict(x_train_adv)

print(f"Acc: {accuracy_score(y_test, y_pred)}, Robust Acc: {accuracy_score(y_test, y_pred_adv)}")

Acc: 0.8947368421052632, Robust Acc: 0.8947368421052632


### Logistic Regression

In [220]:
# Create and fit RandomForestClassifier
model = LogisticRegression(random_state = 0)
model.fit(X_train, y_train)

# Create ART classifier for scikit-learn RandomForestClassifier
art_classifier = SklearnClassifier(model=model)

# Create ART Zeroth Order Optimization attack
zoo = ZooAttack(classifier=art_classifier, binary_search_steps=10, nb_parallel=1)

# Generate adversarial samples with ART Zeroth Order Optimization attack
x_train_adv = zoo.generate(X_test)

C:\Users\ivanr\anaconda3\envs\PhD\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ZOO:   0%|          | 0/114 [00:00<?, ?it/s]

In [221]:
y_pred = model.predict(X_test)
y_pred_adv = model.predict(x_train_adv)

print(f"Acc: {accuracy_score(y_test, y_pred)}, Robust Acc: {accuracy_score(y_test, y_pred_adv)}")

Acc: 0.9649122807017544, Robust Acc: 0.9473684210526315


### KNN

In [226]:
from sklearn.neighbors import KNeighborsClassifier

# Create and fit RandomForestClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)

# Create ART classifier for scikit-learn RandomForestClassifier
art_classifier = SklearnClassifier(model=model)

# Create ART Zeroth Order Optimization attack
# zoo = ZooAttack(classifier=art_classifier, binary_search_steps=10, nb_parallel=1)

zoo = ZooAttack(classifier=art_classifier, confidence=0.0, targeted=False, learning_rate=1e-1, max_iter=100,
                binary_search_steps=20, initial_const=1e-3, abort_early=True, use_resize=False, 
                use_importance=False, nb_parallel=10, batch_size=1, variable_h=0.25)

# Generate adversarial samples with ART Zeroth Order Optimization attack
x_train_adv = zoo.generate(X_test)

ZOO:   0%|          | 0/114 [00:00<?, ?it/s]

In [241]:
y_pred = model.predict(X_test)
y_pred_adv = model.predict(x_train_adv)

print(f"Acc: {accuracy_score(y_test, y_pred)}, Robust Acc: {accuracy_score(y_test, y_pred_adv)}")

Acc: 0.8947368421052632, Robust Acc: 0.8947368421052632


In [242]:
model.score(X_test, y_test), model.score(x_train_adv, y_test)

(0.8947368421052632, 0.8947368421052632)

In [224]:
x_train_adv

array([[1.247e+01, 1.860e+01, 8.109e+01, ..., 1.015e-01, 3.014e-01,
        8.750e-02],
       [1.894e+01, 2.131e+01, 1.236e+02, ..., 1.789e-01, 2.551e-01,
        6.589e-02],
       [1.546e+01, 1.948e+01, 1.017e+02, ..., 1.514e-01, 2.837e-01,
        8.019e-02],
       ...,
       [1.152e+01, 1.493e+01, 7.387e+01, ..., 9.608e-02, 2.664e-01,
        7.809e-02],
       [1.422e+01, 2.785e+01, 9.255e+01, ..., 8.219e-02, 1.890e-01,
        7.796e-02],
       [2.073e+01, 3.112e+01, 1.357e+02, ..., 1.659e-01, 2.868e-01,
        8.218e-02]])

In [225]:
X_test

array([[1.247e+01, 1.860e+01, 8.109e+01, ..., 1.015e-01, 3.014e-01,
        8.750e-02],
       [1.894e+01, 2.131e+01, 1.236e+02, ..., 1.789e-01, 2.551e-01,
        6.589e-02],
       [1.546e+01, 1.948e+01, 1.017e+02, ..., 1.514e-01, 2.837e-01,
        8.019e-02],
       ...,
       [1.152e+01, 1.493e+01, 7.387e+01, ..., 9.608e-02, 2.664e-01,
        7.809e-02],
       [1.422e+01, 2.785e+01, 9.255e+01, ..., 8.219e-02, 1.890e-01,
        7.796e-02],
       [2.073e+01, 3.112e+01, 1.357e+02, ..., 1.659e-01, 2.868e-01,
        8.218e-02]])